# Importando as bibliotecas e construindo o dataframe

In [ ]:
import pandas as pd

In [2]:
# Read the CSV files
fake_df = pd.read_csv('Fake.csv')
true_df = pd.read_csv('True.csv')

# Add a column to indicate if the news is fake or true
fake_df['label'] = 0
true_df['label'] = 1

# Concatenate the two DataFrames
combined_df = pd.concat([fake_df, true_df], ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv('Combined.csv', index=False)

,title,subject,date,label
text,,,,
,627,627,627,627
,4,4,4,4
"(Welcome) to America We hope you enjoy our benefits. Please feel free to NOT assimilate Coming soon: Press 1 for English, 2 for Spanish and 3 for Arabic The Obama Administration is on pace to issue more than a million green cards to migrants from majority-Muslim countries, according to an analysis of Department of Homeland Security data.A chart released by the Senate Subcommittee on Immigration and the National Interest Friday details the surge in immigration to the U.S. from majority-Muslim countries since President Barack Obama took office in 2009.Specifically, in the first six fiscal years of Obama s presidency (FY2009 FY2014), his administration issued 832,014 green cards to migrants majority-Muslim countries, the most of which were issued to migrants from Pakistan (102,000), Iraq (102,000), Bangladesh (90,000), Iran (85,000), Egypt (56,000), and Somalia (37,000).The total 832,014 new permanent residents do not include migrants on temporary, nonimmigrant visas which allow foreign nationals to come to the U.S. temporarily for work, study, tourism and the like. As the subcommittee notes, the number also does not include those migrants who overstayed the terms of their visas.Regardless, as the subcommittee explained in its analysis, the U.S. is playing host to immigrants from majority Muslim countries at an increasing pace.Between FY 2013 and FY 2014, the number of green cards issued to migrants from Muslim-majority countries increased dramatically from 117,423 in FY 2013, to 148,810 in FY 2014, a nearly 27 percent increase. Throughout the Obama Administration s tenure, the United States has issued green cards to an average of 138,669 migrants from Muslim-majority countries per year, meaning that it is nearly certain the United States will have issued green cards to at least 1.1 million migrants from Muslim-majority countries on the President s watch. It has also been reported that migration from Muslim-majority countries represents the fastest growing class of migrants.Green cards, or Lawful Permanent Residency, puts immigrants on the path to citizenship and allows for lifetime residency, federal benefits, and work authorization. Included in the totals are refugees, who are required to apply for a green card after one year of residency in the U.S. Unlike other types of immigrants, refugees are immediately eligible for welfare benefits including Temporary Assistance to Needy Families (TANF), food stamps, and Medicaid. Via: Breitbart News",2,2,2,2
"Delta Air Lines and Bank of America became the first companies to announce they are pulling their sponsorships of a Manhattan-based theater company s portrayal of Julius Caesar as a Donald Trump look-alike in a business suit who gets stabbed to death on stage.Delta and Bank of America both announced their intentions on Sunday. No matter what your political stance may be, the graphic staging of Julius Caesar at this summer s Free Shakespeare in the Park does not reflect Delta Air Lines values, the company s statement said. Their artistic and creative direction crossed the line on the standards of good taste. Bank of America added it was withdrawing funding for the production. The Public Theater chose to present Julius Caesar in such a way that was intended to provoke and offend, the bank said in a tweet. Had this intention been made known to us, we would have decided not to sponsor it. Fox News American Express also took to Twitter and announced their sponsorship doesn t support the play.A Statement from American Express: pic.twitter.com/Ig0Ju3B2dP American Express (@AmericanExpress) June 12, 2017 We would like to clarify that our sponsorship of the Public Theater does not fund the production of Shakespeare in the Park nor do we condone the interpretation of the Julius Caesar play, the financial corporation tweeted.Customers of American Express responded on Twitter:I'm am considering not accepting AE cards at my business a